# Sentiment Analysis - Data Cleaning
# This notebook cleans and preprocesses the raw data


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# ============================================================
# 1. LOAD DATA
# ============================================================

# Load the cleaned_data.csv file
df = pd.read_csv(r'C:\Users\Kirti\OneDrive\Desktop\Data Science D Tech point\Project\sentiment_analysis_project\data\cleaned_data.csv')

print(f"Dataset Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst few rows:")
df.head()


Dataset Shape: (692500, 19)
Columns: ['PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

First few rows:


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,20212,300,4,5.0,1,3.0,1,5.0,1,1,0.0,0,1,9.0,2,0.322,0.208,0.310,0.267
1,20212,249,3,3.0,0,3.0,0,6.0,1,0,0.0,0,1,5.0,0,0.311,0.215,0.292,0.264
2,20203,819,4,3.0,4,3.0,1,4.0,1,0,0.0,0,0,4.0,0,0.297,0.214,0.305,0.264
3,20195,14,26,4.0,0,4.0,1,4.0,1,0,0.0,0,1,4.0,3,0.485,0.172,0.252,0.190
4,20212,907,1,3.0,3,3.0,1,2.0,1,1,0.0,0,1,2.0,1,0.316,0.232,0.285,0.294


In [3]:
# ============================================================
# 2. INITIAL DATA EXPLORATION
# ============================================================

# Display basic information
print("Dataset Info:")
print(df.info())

print("\nDataset Statistics:")
print(df.describe())

print("\nMissing Values:")
print(df.isnull().sum())

print("\nDuplicate Rows:")
print(f"Number of duplicates: {df.duplicated().sum()}")


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 19 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   PERIODO_ACADEMICO            692500 non-null  int64  
 1   E_PRGM_ACADEMICO             692500 non-null  int64  
 2   E_PRGM_DEPARTAMENTO          692500 non-null  int64  
 3   E_VALORMATRICULAUNIVERSIDAD  692500 non-null  float64
 4   E_HORASSEMANATRABAJA         692500 non-null  int64  
 5   F_ESTRATOVIVIENDA            692500 non-null  float64
 6   F_TIENEINTERNET              692500 non-null  int64  
 7   F_EDUCACIONPADRE             692500 non-null  float64
 8   F_TIENELAVADORA              692500 non-null  int64  
 9   F_TIENEAUTOMOVIL             692500 non-null  int64  
 10  E_PRIVADO_LIBERTAD           692466 non-null  float64
 11  E_PAGOMATRICULAPROPIO        692500 non-null  int64  
 12  F_TIENECOMPUTADOR            692500 non-null

In [4]:
# ============================================================
# 3. HANDLE MISSING VALUES
# ============================================================

# Check for missing values in each column
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage': missing_percent
})

print("\nMissing Data Summary:")
print(missing_df[missing_df['Missing Count'] > 0])

# Drop rows with missing values (you can modify this based on your needs)
df_clean = df.dropna()

print(f"\nShape after removing missing values: {df_clean.shape}")



Missing Data Summary:
                    Missing Count  Percentage
E_PRIVADO_LIBERTAD             34     0.00491

Shape after removing missing values: (692466, 19)


In [5]:
# ============================================================
# 4. HANDLE DUPLICATES
# ============================================================

# Remove duplicate rows
df_clean = df_clean.drop_duplicates()

print(f"Shape after removing duplicates: {df_clean.shape}")


Shape after removing duplicates: (692466, 19)


In [6]:
# ============================================================
# 5. TEXT PREPROCESSING (if you have text columns)
# ============================================================

# Function to identify text columns
def identify_text_columns(df):
    text_cols = []
    for col in df.columns:
        if df[col].dtype == 'object':
            # Check if column contains text (average string length > 10)
            avg_length = df[col].astype(str).str.len().mean()
            if avg_length > 10:
                text_cols.append(col)
    return text_cols

text_columns = identify_text_columns(df_clean)
print(f"\nIdentified text columns: {text_columns}")

# Text preprocessing functions
def clean_text(text):
    """Clean and preprocess text data"""
    if pd.isna(text):
        return ""
    
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove numbers (optional - uncomment if needed)
    # text = re.sub(r'\d+', '', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text

def remove_stopwords(text):
    """Remove common stopwords"""
    stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for',
                 'of', 'with', 'is', 'was', 'are', 'were', 'be', 'been', 'being',
                 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'should',
                 'could', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those'}
    
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# Apply text cleaning to identified text columns
if text_columns:
    for col in text_columns:
        print(f"\nCleaning column: {col}")
        df_clean[f'{col}_cleaned'] = df_clean[col].apply(clean_text)
        
        # Optional: Remove stopwords
        # df_clean[f'{col}_no_stopwords'] = df_clean[f'{col}_cleaned'].apply(remove_stopwords)
        
        print(f"Example before cleaning:\n{df_clean[col].iloc[0]}")
        print(f"\nExample after cleaning:\n{df_clean[f'{col}_cleaned'].iloc[0]}")



Identified text columns: []


In [7]:
# ============================================================
# 6. DATA TYPE CONVERSION
# ============================================================

# Convert date columns if any
date_patterns = ['periodo', 'date', 'fecha', 'time', 'timestamp']
for col in df_clean.columns:
    if any(pattern in col.lower() for pattern in date_patterns):
        try:
            df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
            print(f"Converted {col} to datetime")
        except:
            print(f"Could not convert {col} to datetime")

Converted PERIODO_ACADEMICO to datetime


In [8]:
# ============================================================
# 7. OUTLIER DETECTION (for numerical columns)
# ============================================================

# Identify numerical columns
numerical_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()

print(f"\nNumerical columns: {numerical_cols}")

# Function to detect outliers using IQR method
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Check for outliers in numerical columns
print("\nOutlier Detection:")
for col in numerical_cols[:5]:  # Check first 5 numerical columns
    outliers, lower, upper = detect_outliers_iqr(df_clean, col)
    print(f"\n{col}:")
    print(f"  Outliers: {len(outliers)} ({len(outliers)/len(df_clean)*100:.2f}%)")
    print(f"  Range: [{lower:.2f}, {upper:.2f}]")



Numerical columns: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

Outlier Detection:

E_PRGM_ACADEMICO:
  Outliers: 0 (0.00%)
  Range: [-551.00, 1457.00]

E_PRGM_DEPARTAMENTO:
  Outliers: 0 (0.00%)
  Range: [-15.50, 36.50]

E_VALORMATRICULAUNIVERSIDAD:
  Outliers: 0 (0.00%)
  Range: [-3.50, 8.50]

E_HORASSEMANATRABAJA:
  Outliers: 0 (0.00%)
  Range: [-3.50, 8.50]

F_ESTRATOVIVIENDA:
  Outliers: 36213 (5.23%)
  Range: [0.50, 4.50]


In [9]:
# ============================================================
# 8. FEATURE ENGINEERING
# ============================================================

# Add text length feature if text columns exist
if text_columns:
    for col in text_columns:
        df_clean[f'{col}_length'] = df_clean[col].astype(str).str.len()
        df_clean[f'{col}_word_count'] = df_clean[col].astype(str).str.split().str.len()


In [10]:
# ============================================================
# 9. DATA VALIDATION
# ============================================================

print("\n" + "="*60)
print("DATA VALIDATION SUMMARY")
print("="*60)

print(f"\nOriginal dataset shape: {df.shape}")
print(f"Cleaned dataset shape: {df_clean.shape}")
print(f"Rows removed: {len(df) - len(df_clean)}")
print(f"Percentage retained: {len(df_clean)/len(df)*100:.2f}%")

print("\nFinal Data Types:")
print(df_clean.dtypes)

print("\nFinal Missing Values:")
print(df_clean.isnull().sum().sum())


DATA VALIDATION SUMMARY

Original dataset shape: (692500, 19)
Cleaned dataset shape: (692466, 19)
Rows removed: 34
Percentage retained: 100.00%

Final Data Types:
PERIODO_ACADEMICO              datetime64[ns]
E_PRGM_ACADEMICO                        int64
E_PRGM_DEPARTAMENTO                     int64
E_VALORMATRICULAUNIVERSIDAD           float64
E_HORASSEMANATRABAJA                    int64
F_ESTRATOVIVIENDA                     float64
F_TIENEINTERNET                         int64
F_EDUCACIONPADRE                      float64
F_TIENELAVADORA                         int64
F_TIENEAUTOMOVIL                        int64
E_PRIVADO_LIBERTAD                    float64
E_PAGOMATRICULAPROPIO                   int64
F_TIENECOMPUTADOR                       int64
F_EDUCACIONMADRE                      float64
RENDIMIENTO_GLOBAL                      int64
INDICADOR_1                           float64
INDICADOR_2                           float64
INDICADOR_3                           float64
INDICADO

In [11]:
# ============================================================
# 10. SAVE CLEANED DATA
# ============================================================

# Save cleaned dataset
output_path = '../data/cleaned_data_processed.csv'
df_clean.to_csv(output_path, index=False)

print(f"\nCleaned data saved to: {output_path}")

# Also save a sample for quick reference
sample_path = '../data/sample_cleaned_data.csv'
df_clean.head(100).to_csv(sample_path, index=False)

print(f"Sample data saved to: {sample_path}")

print("\n✅ Data cleaning completed successfully!")

# Display final preview
print("\nFinal Dataset Preview:")
df_clean.head(10)


Cleaned data saved to: ../data/cleaned_data_processed.csv
Sample data saved to: ../data/sample_cleaned_data.csv

✅ Data cleaning completed successfully!

Final Dataset Preview:


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,1970-01-01 00:00:00.000020212,300,4,5.0,1,3.0,1,5.0,1,1,0.0,0,1,9.0,2,0.322,0.208,0.310,0.267
1,1970-01-01 00:00:00.000020212,249,3,3.0,0,3.0,0,6.0,1,0,0.0,0,1,5.0,0,0.311,0.215,0.292,0.264
2,1970-01-01 00:00:00.000020203,819,4,3.0,4,3.0,1,4.0,1,0,0.0,0,0,4.0,0,0.297,0.214,0.305,0.264
3,1970-01-01 00:00:00.000020195,14,26,4.0,0,4.0,1,4.0,1,0,0.0,0,1,4.0,3,0.485,0.172,0.252,0.190
4,1970-01-01 00:00:00.000020212,907,1,3.0,3,3.0,1,2.0,1,1,0.0,0,1,2.0,1,0.316,0.232,0.285,0.294
5,1970-01-01 00:00:00.000020203,811,1,6.0,1,5.0,1,8.0,1,1,0.0,0,1,4.0,2,0.553,0.142,0.248,0.175
6,1970-01-01 00:00:00.000020183,455,16,3.0,3,2.0,1,7.0,1,1,0.0,1,1,6.0,3,0.242,0.299,0.267,0.308
7,1970-01-01 00:00:00.000020183,94,4,2.0,2,2.0,1,1.0,1,0,0.0,1,0,3.0,1,0.238,0.316,0.286,0.290
8,1970-01-01 00:00:00.000020212,452,3,5.0,1,1.0,1,4.0,1,1,0.0,1,1,7.0,1,0.147,0.407,0.212,0.328
9,1970-01-01 00:00:00.000020183,14,1,3.0,4,5.0,1,9.0,1,1,0.0,1,1,9.0,3,0.535,0.122,0.257,0.167
